In [26]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import truncnorm
import time
import csv


def get_s_cond_t_params(player_1_sigma, player_2_sigma, player_1_mean, player_2_mean, t_i):
    beta = 3  # från uppgiften

    # Covariance calculations
    covariance_t_given_s = beta
    covariance_ss = np.array([[player_1_sigma, 0], [0, player_2_sigma]])
    A = np.array([1, -1]).reshape([1, 2])
    inv_covariance_ss = np.linalg.inv(covariance_ss)
    ACA = np.matmul(A.T, A) * (1 / covariance_t_given_s)
    covariance_s_cond_t = np.linalg.inv(inv_covariance_ss + ACA)

    # Mean calculations
    player_means = np.array([player_1_mean, player_2_mean]).reshape([2, 1])
    a = covariance_s_cond_t
    b = np.matmul(inv_covariance_ss, player_means)
    c = A.T * (1 / covariance_t_given_s) * t_i
    mean_s_cond_t = np.matmul(a, b + c)

    return mean_s_cond_t, covariance_s_cond_t


def P_s_cond_t(t_i, player_1_mean, player_1_sigma, player_2_mean, player_2_sigma):
    # player_1_mean = 25; player_2_mean = 25; player_1_sigma = 8.3**2; player_2_sigma = 8.3**2

    mean_s_cond_t, cov_s_cond_t = get_s_cond_t_params(player_1_mean=player_1_mean, player_1_sigma=player_1_sigma,
                                                      player_2_mean=player_2_mean, player_2_sigma=player_2_sigma,
                                                      t_i=t_i)

    return np.random.multivariate_normal(mean=mean_s_cond_t.reshape(2), cov=cov_s_cond_t,
                                         check_valid='warn', tol=1e-8)


def P_t_cond_s(s_i, t_game):
    s_diff = s_i[0] - s_i[1]
    beta = 3
    t_sigma = beta

    if t_game > 0:  # case for when y=1
        a, b = (0 - s_diff) / t_sigma, np.inf
        t = truncnorm.rvs(a, b) * t_sigma + s_diff
        return t
    elif t_game < 0:  # case for when y=-1
        a, b = -np.inf, (0 - s_diff) / t_sigma
        t = truncnorm.rvs(a, b) * t_sigma + s_diff
        return t
    else:
        print("ERROR, TIES PRESENTLY NOT ALLOWED")


        
def gibbs_sampler(L, player_1_stats, player_2_stats, t_game):
    player_1_mean, player_1_sigma = player_1_stats
    player_2_mean, player_2_sigma = player_2_stats
    
    s_i = [player_1_mean, player_2_mean]

    t_obs = np.zeros(L)
    s_obs = np.zeros([L, 2])

    for i in range(L):
        t_i_plus_1 = P_t_cond_s(s_i, t_game=t_game)
        s_i_plus_1 = P_s_cond_t(t_i_plus_1, player_1_mean, player_1_sigma, player_2_mean, player_2_sigma)

        t_obs[i] = t_i_plus_1
        s_obs[i, :] = s_i_plus_1

        s_i = s_i_plus_1
        # plt.scatter(s_obs[:, 0], s_obs[:, 1])
        # plt.pause(0.1)
    # plt.show()

    # plt.plot(s_obs[:, 0]); plt.plot(s_obs[:, 1]); plt.show()
    return s_obs, t_obs



def player_stats_estimate_from_obs(s_obs):
    player_1_stats_estimate = [np.mean(s_obs[:, 0]), np.var(s_obs[:, 0])] # [mean, variance] of samples
    player_2_stats_estimate = [np.mean(s_obs[:, 1]), np.var(s_obs[:, 1])] # [mean, variance] of samples
    
    return player_1_stats_estimate, player_2_stats_estimate


    

def make_stats_dictionary(filename, stats_dictionary, printable):
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        mean = 100; variance = (100 / 3) ** 2  # TrueSkill prior parameters before any games
        gamelist = []
        for row in reader:
            # Create dictionary (=map) with keyword 'team' and value [mean, variance]
            stats_dictionary[row['team1']] = [mean, variance]
            # Add teams and result to list
            gamelist.append([row['team1'], row['team2'], int(row['score1']) - int(row['score2'])])

            if printable == 1:  # Print the whole list
                print(f"{row['team1']} vs {row['team2']}: {row['score1']}-{row['score2']}")  # Access by column header
                      
    #stats_dictionary = random.shuffle(stats_dictionary, random)

    return stats_dictionary, gamelist



# Ranking by mean (should perhaps be improved to mean - 3 * sigma)
def ranking(stats_dictionary):
    sorted_teams = sorted(stats_dictionary.items(), key=lambda x: x[1], reverse=True)
    print("\nList of teams ranked by mean skill in descending order:\n")
    for i in sorted_teams:
        print(i[0], i[1])

        
def predict_winner(team1_string, team2_string, stats_dictionary):
    diff = stats_dictionary[team1_string][0] - stats_dictionary[team2_string][0] 
    #print('statdic team1 : ', stats_dictionary[team1_string][0])
    #print('statdic team2 : ', stats_dictionary[team2_string][0])
    print('\t diff: ', diff)
    
    if diff > 0:
        return 1
    elif diff < 0:
        return -1
    else:
        return 1
            


In [27]:
# Make dictionary of team stats (mean and variance) & list of all games with result
stats_dictionary, result_list = make_stats_dictionary('SerieA.csv', stats_dictionary={}, printable=0)


In [28]:
stats_dictionary

{'Chievo': [100, 1111.1111111111113],
 'Lazio': [100, 1111.1111111111113],
 'Torino': [100, 1111.1111111111113],
 'Sassuolo': [100, 1111.1111111111113],
 'Parma': [100, 1111.1111111111113],
 'Empoli': [100, 1111.1111111111113],
 'Bologna': [100, 1111.1111111111113],
 'Atalanta': [100, 1111.1111111111113],
 'Juventus': [100, 1111.1111111111113],
 'Napoli': [100, 1111.1111111111113],
 'Spal': [100, 1111.1111111111113],
 'Udinese': [100, 1111.1111111111113],
 'Inter': [100, 1111.1111111111113],
 'Genoa': [100, 1111.1111111111113],
 'Frosinone': [100, 1111.1111111111113],
 'Fiorentina': [100, 1111.1111111111113],
 'Cagliari': [100, 1111.1111111111113],
 'Roma': [100, 1111.1111111111113],
 'Milan': [100, 1111.1111111111113],
 'Sampdoria': [100, 1111.1111111111113]}

In [29]:
result_list

[['Chievo', 'Juventus', -1],
 ['Lazio', 'Napoli', -1],
 ['Torino', 'Roma', -1],
 ['Sassuolo', 'Inter', 1],
 ['Parma', 'Udinese', 0],
 ['Empoli', 'Cagliari', 2],
 ['Bologna', 'Spal', -1],
 ['Atalanta', 'Frosinone', 4],
 ['Juventus', 'Lazio', 2],
 ['Napoli', 'Milan', 1],
 ['Spal', 'Parma', 1],
 ['Udinese', 'Sampdoria', 1],
 ['Inter', 'Torino', 0],
 ['Genoa', 'Empoli', 1],
 ['Frosinone', 'Bologna', 0],
 ['Fiorentina', 'Chievo', 5],
 ['Cagliari', 'Sassuolo', 0],
 ['Roma', 'Atalanta', 0],
 ['Milan', 'Roma', 1],
 ['Bologna', 'Inter', -3],
 ['Parma', 'Juventus', -1],
 ['Fiorentina', 'Udinese', 1],
 ['Torino', 'Spal', 1],
 ['Sassuolo', 'Genoa', 2],
 ['Sampdoria', 'Napoli', 3],
 ['Lazio', 'Frosinone', 1],
 ['Chievo', 'Empoli', 0],
 ['Atalanta', 'Cagliari', -1],
 ['Inter', 'Parma', -1],
 ['Napoli', 'Fiorentina', 1],
 ['Frosinone', 'Sampdoria', -5],
 ['Roma', 'Chievo', 0],
 ['Udinese', 'Torino', 0],
 ['Juventus', 'Sassuolo', 1],
 ['Genoa', 'Bologna', 1],
 ['Empoli', 'Lazio', -1],
 ['Cagliari', 'M

In [22]:

# Make dictionary of team stats (mean and variance) & list of all games with result
stats_dictionary, result_list = make_stats_dictionary('SerieA.csv', stats_dictionary={}, printable=0)

correct_predictions = 0
nr_of_draws = 0

''' 
UNCOMMENT THIS SECTION TO GET A RANDOMIXED ORDERING OF THE MATCHES. 
# 
If you want to randomize

'''

#x = np.random.choice(range(len(result_list)), size=len(result_list), replace=False)
#randomized_list = []
#for i in x:
#    randomized_list.append(result_list[i])
#result_list = randomized_list[:250]

result_list = result_list[:250]
for i in range(len(result_list)):
    # result_list holds [team1, team2, result = score1 - score2]
    print(f"\nResult game {i}:   {result_list[i]}")
    team1 = result_list[i][0]
    team2 = result_list[i][1]
    
    print('team1: ', team1)
    print('team2: ', team2)
    result = result_list[i][2]
    
    # stats_dictionary with keyword 'teamname' and value [mean, variance]
    print(f"Stats before game: {stats_dictionary[team1]}, {stats_dictionary[team2]}")

    if result == 0:  # ignore tied games for now
        print("Game ignored due to tie")
        print(f"Stats after game:  {stats_dictionary[team1]}, {stats_dictionary[team2]}")
        nr_of_draws += 1
        print("Draws: ", nr_of_draws)
    else:
    
        # prediktera mat resultat och spara prediktion
        pred_result = predict_winner(team1, team2, stats_dictionary)
        
        if np.sign(result) == np.sign(pred_result):
                correct_predictions += 1
                print("Correct pred: ", correct_predictions)
        
        print('true result: ', result)
        print('pred result: ', pred_result)
                
        s_obs, t_obs =  gibbs_sampler(L=5000,
                                      player_1_stats=stats_dictionary[team1], 
                                      player_2_stats=stats_dictionary[team2],
                                      t_game=result)
        
        player_1_stats_posterior, player_2_stats_posterior = player_stats_estimate_from_obs(s_obs[3000:, :])
        
        
        # Update team stats so posterior makes new prior
        stats_dictionary[team1] = player_1_stats_posterior
        stats_dictionary[team2] = player_2_stats_posterior


        print(f"Stats AFTER game: {stats_dictionary[team1]}, {stats_dictionary[team2]}")

print()
print("correct_predictions ", correct_predictions)
print("reslist: ", len(result_list))
print("draws: ", nr_of_draws)
print('performance: ', correct_predictions / (len(result_list)-nr_of_draws))



Result game 0:   ['Chievo', 'Juventus', -1]
team1:  Chievo
team2:  Juventus
Stats before game: [100, 1111.1111111111113], [100, 1111.1111111111113]
	 diff:  0
true result:  -1
pred result:  1
Stats AFTER game: [73.42557273293133, 972.2330540924062], [126.07058175528802, 952.3671564748918]

Result game 1:   ['Lazio', 'Napoli', -1]
team1:  Lazio
team2:  Napoli
Stats before game: [100, 1111.1111111111113], [100, 1111.1111111111113]
	 diff:  0
true result:  -1
pred result:  1
Stats AFTER game: [86.04689855378842, 649.7298104648073], [113.60744614183425, 627.782680320441]

Result game 2:   ['Torino', 'Roma', -1]
team1:  Torino
team2:  Roma
Stats before game: [100, 1111.1111111111113], [100, 1111.1111111111113]
	 diff:  0
true result:  -1
pred result:  1
Stats AFTER game: [74.57746680112035, 736.9440619476289], [127.73982171326978, 706.0123237519833]

Result game 3:   ['Sassuolo', 'Inter', 1]
team1:  Sassuolo
team2:  Inter
Stats before game: [100, 1111.1111111111113], [100, 1111.11111111111

In [23]:
# Q5 NOTES. 

'''
#Running this procedure again resulted in a similar raking with only slight differences in the final ordering of the teams, when ordered by thier estimated skill.
# In figure xxx we have present the teams ranked accorind to their estimaetd skills for two runs. 


Q:  How can you interpret the variance of the final skills? 
Since TrueSkill beased models are baysian in anture when it comes to a players skill, the variances accosiated with thier skill reflects the models/systsm degree of cetianty
of thier true skills. In other words, the variance is a measure of how sure we are of the skill mean estimate. 

'''




'\n#Running this procedure again resulted in a similar raking with only slight differences in the final ordering of the teams, when ordered by thier estimated skill.\n# In figure xxx we have present the teams ranked accorind to their estimaetd skills for two runs. \n\n\nQ:  How can you interpret the variance of the final skills? \nSince TrueSkill beased models are baysian in anture when it comes to a players skill, the variances accosiated with thier skill reflects the models/systsm degree of cetianty\nof thier true skills. In other words, the variance is a measure of how sure we are of the skill mean estimate. \n\n'

In [24]:
stats_dictionary


{'Chievo': [64.53452699944494, 417.935443830805],
 'Lazio': [97.60548099135899, 136.05234412787337],
 'Torino': [124.38365638697235, 63.795434509529194],
 'Sassuolo': [81.91038034499981, 89.0748771822913],
 'Parma': [88.74989181694053, 31.4122006063775],
 'Empoli': [95.0686117997742, 80.6546960428363],
 'Bologna': [71.8835812103959, 77.89940242616134],
 'Atalanta': [109.47440262524823, 78.39251610133323],
 'Juventus': [451.24878613174917, 774.2229975805711],
 'Napoli': [140.71074234211864, 427.9935880328483],
 'Spal': [100.67570541486951, 73.63173302159734],
 'Udinese': [93.14063061463618, 92.72145342416465],
 'Inter': [91.71937571212766, 27.08753660981291],
 'Genoa': [115.0708091013272, 58.30903891668599],
 'Frosinone': [95.98449509064878, 77.67911879023922],
 'Fiorentina': [113.2025322048611, 50.47006607358613],
 'Cagliari': [98.50289285590306, 35.44674623732617],
 'Roma': [128.80175705068427, 135.64100504235657],
 'Milan': [143.84661235276502, 267.20087690669766],
 'Sampdoria': [96.

In [25]:
ranking(stats_dictionary)
'fjärde (randomiserad) körningen'


List of teams ranked by mean skill in descending order:

Juventus [451.24878613174917, 774.2229975805711]
Milan [143.84661235276502, 267.20087690669766]
Napoli [140.71074234211864, 427.9935880328483]
Roma [128.80175705068427, 135.64100504235657]
Torino [124.38365638697235, 63.795434509529194]
Genoa [115.0708091013272, 58.30903891668599]
Fiorentina [113.2025322048611, 50.47006607358613]
Atalanta [109.47440262524823, 78.39251610133323]
Spal [100.67570541486951, 73.63173302159734]
Cagliari [98.50289285590306, 35.44674623732617]
Lazio [97.60548099135899, 136.05234412787337]
Sampdoria [96.04309045264434, 44.5410460729608]
Frosinone [95.98449509064878, 77.67911879023922]
Empoli [95.0686117997742, 80.6546960428363]
Udinese [93.14063061463618, 92.72145342416465]
Inter [91.71937571212766, 27.08753660981291]
Parma [88.74989181694053, 31.4122006063775]
Sassuolo [81.91038034499981, 89.0748771822913]
Bologna [71.8835812103959, 77.89940242616134]
Chievo [64.53452699944494, 417.935443830805]


'fjärde (randomiserad) körningen'

In [10]:
# Q4a 
nr_of_samples = 50000

player_1_mean = 100
player_1_var = 10
player_1_stats = [player_1_mean, player_1_var]

player_2_mean = 100
player_2_var = 10
player_2_stats = [player_2_mean, player_2_var]

match_outcome = 1 # player 1 wins

s_obs_1, t_obs_1 = gibbs_sampler(L=nr_of_samples,
                             player_1_stats=player_1_stats,
                             player_2_stats=player_2_stats,
                             t_game=match_outcome)

s_obs_2, t_obs_2 = gibbs_sampler(L=nr_of_samples,
                             player_1_stats=player_1_stats,
                             player_2_stats=player_2_stats,
                             t_game=match_outcome)

s_obs_3, t_obs_3 = gibbs_sampler(L=nr_of_samples,
                             player_1_stats=player_1_stats,
                             player_2_stats=player_2_stats,
                             t_game=match_outcome)


In [ ]:

#plotting mean of all observed values up until the i'th obs

mean_vector_1 = np.cumsum(s_obs_1[:,0])/(np.array(list(range(len(s_obs_1[:,0]))))+1)
mean_vector_2 = np.cumsum(s_obs_2[:,0])/(np.array(list(range(len(s_obs_2[:,0]))))+1)
mean_vector_3 = np.cumsum(s_obs_3[:,0])/(np.array(list(range(len(s_obs_3[:,0]))))+1)

plt.figure(figsize=[10,5])
plt.grid()
plt.plot(mean_vector_1, color='blue')
plt.plot(mean_vector_2, color='green')
plt.plot(mean_vector_3, color='red')

# Notes: All three samples tend to converge on the same value eventually with seemingly smaller and smaller variance as i-> inf
# We observe that given only a single run of 50000 gibbs samples would lead us to a lower burn in (around 1000 based only on 
# the 'blue observations'). This makes us conclude that a suitble burn in rate should be estimated by analysing serveral runs and 
# at several length to be assured of both converegance to the same dist (based on some param estimate from the sample) but also determine a 
# global suitable burn-in value. 


In [ ]:
# Q.4a

plt.grid()
#plotting mean of all observed values up until the i'th obs

plt.plot(mean_vector_1[:20000], color='blue')
plt.plot(mean_vector_2[:20000], color='green')
plt.plot(mean_vector_3[:20000], color='red')


plt.title("mean up until ith gibbs sample/observation")
plt.xlabel("'i'th gibbs sample/observation ")
plt.ylabel("mean up unitl i'th")


'''
Notes: 
Suitable burn-in value between 3000-5000.
For sake of computational resource efficienty, we choose 3000 as our burn-in value
''' 

In [ ]:
# Q.4a
burn_in_indx = 3000
nr_samples_q4c = 15000

time.tik
s_obs, t_obs = gibbs_sampler(L=nr_samples_q4c,
                             player_1_stats=player_1_stats,
                             player_2_stats=player_2_stats,
                             t_game=match_outcome)

s_obs = s_obs[burn_in_indx:, :] # discard inital B samples due to burn in. 


player_1_stats_estimate_1, player_2_stats_estimate = player_stats_estimate_from_obs(s_obs=s_obs[burn_in_indx:burn_in_indx+100,:])
player_1_stats_estimate_2, player_2_stats_estimate = player_stats_estimate_from_obs(s_obs=s_obs[burn_in_indx:burn_in_indx+500, :])
player_1_stats_estimate_3, player_2_stats_estimate = player_stats_estimate_from_obs(s_obs=s_obs[burn_in_indx:burn_in_indx+2500, :])
player_1_stats_estimate_4, player_2_stats_estimate = player_stats_estimate_from_obs(s_obs=s_obs[burn_in_indx:burn_in_indx+10000, :])


p1_mu_est_1 = player_1_stats_estimate_1[0]
p1_var_est_1 = player_1_stats_estimate_1[1]
p1_sigma_est_1 = np.sqrt(p1_var_est_1)

p1_mu_est_2 = player_1_stats_estimate_2[0]
p1_var_est_2 = player_1_stats_estimate_2[1]
p1_sigma_est_2 = np.sqrt(p1_var_est_2)

p1_mu_est_3 = player_1_stats_estimate_3[0]
p1_var_est_3 = player_1_stats_estimate_3[1]
p1_sigma_est_3 = np.sqrt(p1_var_est_3)

p1_mu_est_4 = player_1_stats_estimate_4[0]
p1_var_est_4 = player_1_stats_estimate_4[1]
p1_sigma_est_4 = np.sqrt(p1_var_est_4)


x1 = np.linspace(p1_mu_est_1 - 3 * p1_sigma_est_1, p1_mu_est_1 + 3 * p1_sigma_est_1, 100)
x2 = np.linspace(p1_mu_est_2 - 3 * p1_sigma_est_2, p1_mu_est_2 + 3 * p1_sigma_est_2, 100)
x3 = np.linspace(p1_mu_est_3 - 3 * p1_sigma_est_3, p1_mu_est_3 + 3 * p1_sigma_est_3, 100)
x4 = np.linspace(p1_mu_est_4 - 3 * p1_sigma_est_4, p1_mu_est_4 + 3 * p1_sigma_est_4, 100)

plt.figure(figsize=[20,5])
plt.subplot(1,4, 1) 
plt.hist(s_obs[burn_in_indx:burn_in_indx+100, 0], bins=50, density=True) #P(S1 \Y=1)
plt.plot(x1, stats.norm.pdf(x1, loc = p1_mu_est_1, scale = p1_sigma_est_1))

plt.subplot(1,4, 2) 
plt.hist(s_obs[burn_in_indx:burn_in_indx+500, 0], bins=50, density=True) #P(S1 \Y=1)
plt.plot(x2, stats.norm.pdf(x2, loc = p1_mu_est_2, scale = p1_sigma_est_2))

plt.subplot(1,4, 3) 
plt.hist(s_obs[burn_in_indx:burn_in_indx+3000, 0], bins=50, density=True) #P(S1 \Y=1)
plt.plot(x3, stats.norm.pdf(x3, loc = p1_mu_est_3, scale = p1_sigma_est_3))

plt.subplot(1,4, 4) 
plt.hist(s_obs[burn_in_indx:burn_in_indx+10000, 0], bins=50, density=True) #P(S1 \Y=1)
plt.plot(x4, stats.norm.pdf(x4, loc = p1_mu_est_4, scale = p1_sigma_est_4))



In [ ]:
# Skill scatter plot for both players 
plt.figure
plt.scatter(s_obs[:,0], s_obs[:,1])
plt.scatter(s_obs[burn_in_indx:,0], s_obs[burn_in_indx:, 1], c='orange')
plt.xlabel('player 1 skill estimate (arbitrary units)')
plt.ylabel('player 2 skill estimate (arbitrary units)')
plt.legend({'asd', 'sdsfsd', 'asuhh '})


In [ ]:
'''

if __name__ == '__main__':
    # Make dictionary of team stats (mean and variance) & list of all games with result
    stats_dictionary, result_list = make_stats_dictionary('SerieA.csv', stats_dictionary={}, printable=0)
    print(stats_dictionary)
    
    
    # If you want to randomize
    x = np.random.choice(range(len(result_list)), size=len(result_list), replace=False)
    randomized_list = []
    for i in x:
        randomized_list.append(result_list[i])
    result_list = randomized_list

    # Iterate game list using Gibbs-sampler to estimate posterior for skills, using them as new priors
    for i in range(len(result_list)):
        # result_list holds [team1, team2, result = score1 - score2]
        print(f"\nResult game {i}:   {result_list[i]}")
        team1 = result_list[i][0]
        team2 = result_list[i][1]
        result = result_list[i][2]
        # stats_dictionary with keyword 'teamname' and value [mean, variance]
        print(f"Stats before game: {stats_dictionary[team1]}, {stats_dictionary[team2]}")

        if result == 0:  # ignore tied games for now
            print("Game ignored due to tie")
            print(f"Stats after game:  {stats_dictionary[team1]}, {stats_dictionary[team2]}")

        else:
            
            player_1_stats_posterior, player_2_stats_posterior = gibbs_sampler(L=100,
                                                                               player_1_stats=stats_dictionary[team1],
                                                                               player_2_stats=stats_dictionary[team2],
                                                                               t_game=result)
            # Update team stats so posterior makes new prior
            stats_dictionary[team1] = player_1_stats_posterior
            stats_dictionary[team2] = player_2_stats_posterior
            
            
            print(f"Stats after game:  {player_1_stats_posterior}, {player_2_stats_posterior}")


print(stats_dictionary)
    ranking(stats_dictionary)

'''
